# MiDaS

### Описание

[MiDaS](https://arxiv.org/abs/1907.01341) позволяет вычислять карту глубины по изображению. Этот блокнот даёт возможность запускать несколько различных моделей (например, `v2.1` / `v3 Hybrid` / `v3 Large`) на вашем наборе изображений. 

Перед использованием создайте на своём гугл-диске папку `midas-source`, в которую поместите изображения, карты глубин которых вы хотите получить. Не используйте в наименовании файлов пробелы. После обработки результаты будут помещены в папку `midas-result` на вашем хранилище.

Этот блокнот рассчитан на работу в Google Colab, чтобы использовать вычислительные мощности облака, а не вашего компьютера. Если вы всё ещё не там, необходимо сделать следующее:

1. Перейти по ссылке https://colab.research.google.com/
2. Выбрать пункт "GitHub"
3. Вставить туда ссылку на этот блокнот (на текущий момент это https://github.com/Sertyhopss/midas-notebook/blob/master/MiDaS.ipynb)
4. Выбрать файл MiDaS.ipynb

или, если файл хранится у вас локально:

1. Перейти по ссылке https://colab.research.google.com/
2. Выбрать пункт "Загрузить"
3. Вставить в открывшееся окно текущий файл

Приятного использования!

###Установка необходимых зависимостей

In [ ]:
pip install timm opencv-python torch

### Импорт зависимостей и получение разрешения на использование google drive

In [ ]:
import cv2
import torch
import urllib.request

import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
from pathlib import Path

drive.mount('/content/drive', force_remount=True)
DRIVE = Path('/content/drive/MyDrive')

### Определение источника изображений

При наличии папки `midas-source` в вашем хранилище, изображения берутся оттуда.

В обратном случае изображения берутся из сети. Если вы хотите добавить изображения по URL, заполните массив files словарями следующей структуры, как описано в примере: 

`{"url": *ссылка на изображение*, "name": *имя файла с расширением*}`

In [ ]:
supported_formats = ['.png', '.jpg', '.jpeg', '.bmp']

if (DRIVE / 'midas-source').is_dir():
  files = [f for f in (DRIVE / 'midas-source').iterdir() if f.suffix in supported_formats]
  src = [str(f) for f in list(files)]

else:
  files = [
      {"url": "https://github.com/pytorch/hub/raw/master/images/dog.jpg", "name": "dog.jpg"}  # example
  ]
  src = []
  for f in files:
    urllib.request.urlretrieve(f["url"], f["name"])
    src.append(f["name"])

print(f'total images: {len(src)}')

### Выбор модели

Вы можете выбрать любую модель из списка:

* DPT_Large
* DPT_Hybrid
* MiDaS_small

Для сравнения моделей смотрите [https://github.com/intel-isl/MiDaS/#Accuracy](https://github.com/intel-isl/MiDaS/#Accuracy)

In [ ]:
model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)

Используем ГПУ, если такая возможность имеется

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

Загружаем необходимый для моделей трансформер



In [ ]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Функция для записи карты глубины. Преобразуем к .png

In [70]:
def write_depth(path, depth):

    depth_min = depth.min()
    depth_max = depth.max()

    max_val = 65535 # uint16 max value

    if depth_max - depth_min > np.finfo("float").eps:
        out = max_val * (depth - depth_min) / (depth_max - depth_min)
    else:
        out = 0

    path = path.replace(Path(path).suffix, '.png')

    cv2.imwrite(path, out.astype("uint16"))
    
    return

### Обработка

Выполняем преобразование для каждого входного изображения и сохраняем на диск

In [ ]:
res_path = (DRIVE / 'midas-result')
if not res_path.is_dir():
  res_path.mkdir()

for filename in src:

  try:
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    input_batch = transform(img).to(device)

    with torch.no_grad():
      prediction = midas(input_batch)

      prediction = torch.nn.functional.interpolate(
          prediction.unsqueeze(1),
          size=img.shape[:2],
          mode="bicubic",
          align_corners=False,
      ).squeeze()

    output = prediction.cpu().numpy()

    out_file = filename.replace('midas-source', 'midas-result')
    write_depth(out_file, output)
    print(f'{filename} successfully processed.')

  except Exception as e:
    print(e)
    print(f'{filename} processing failed.')

### References
[Towards Robust Monocular Depth Estimation: Mixing Datasets for Zero-shot Cross-dataset Transfer](https://arxiv.org/abs/1907.01341)

[Vision Transformers for Dense Prediction](https://arxiv.org/abs/2103.13413)

Please cite our papers if you use our models.